In [1]:
%reload_ext autoreload

%autoreload 2

import sys
sys.path.append('../models')
sys.path.append('../utils')
sys.path.append('../../config')
from constants import *

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE


## Tratamiento para contrarestar el desbalance de clases

### Modificar tamaño del dataset manteniendo relacion de 10:1

In [3]:
import pandas as pd

# Leer el dataset
df = pd.read_csv(INITIAL_DATASET_ROUTE)

# Suponemos que la columna de clase se llama "Class"
# (ajusta este nombre según tu dataset)
fraud_df = df[df["Class"] == 1]
no_fraud_df = df[df["Class"] == 0]

# Calcular la cantidad de fraudes y la cantidad deseada de no fraudes
n_fraud = len(fraud_df)
n_no_fraud = min(len(no_fraud_df), 10 * n_fraud)

# Seleccionar aleatoriamente el subconjunto de "No fraude"
no_fraud_subset = no_fraud_df.sample(n=n_no_fraud, random_state=42)

# Unir ambos subconjuntos
balanced_df = pd.concat([fraud_df, no_fraud_subset])

# Mezclar las filas para evitar orden sesgado
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
# Guardar el nuevo dataset balanceado

print(f"Datos originales: {len(df)}")
print(f"Fraude: {n_fraud}, No fraude original: {len(no_fraud_df)}, No fraude usado: {n_no_fraud}")
print(f"Nuevo tamaño total: {len(balanced_df)}")


Datos originales: 284807
Fraude: 492, No fraude original: 284315, No fraude usado: 4920
Nuevo tamaño total: 5412


### Manejar desbalance de clases y separar train y test

In [4]:
import pandas as pd

# ========= PARÁMETROS =========
TEST_SIZE = 0.30
SMOTE_RATIO = 0.5         # 1.0 = 1:1 ; 0.5 = 1 fraude por cada 2 no fraude
# ==============================
LABEL_COL = "Class"
# 1) Cargar
df = balanced_df

# 2) Separar X e y (todo numérico)
X = df.drop(columns=[LABEL_COL])
y = df[LABEL_COL].astype(int)


# 3) Split 70/30 estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=SEED
)

# 4) Escalado (fit SOLO en train)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 5) SMOTE SOLO en train (ya escalado)
smote = SMOTE(sampling_strategy=SMOTE_RATIO, random_state=SEED)
X_train_sm, y_train_sm = smote.fit_resample(X_train_scaled, y_train)

# 6) Reconstruir DataFrames y guardar
train_df = pd.DataFrame(X_train_sm, columns=X.columns)
test_df  = pd.DataFrame(X_test_scaled, columns=X.columns)
train_df[LABEL_COL] = y_train_sm.values
test_df[LABEL_COL]  = y_test.values

train_df.to_csv(TRAIN_DATASET_ROUTE, index=False)
test_df.to_csv(TEST_DATASET_ROUTE,  index=False)

# 7) Reporte rápido
def dist(yv):
    vals, cnts = np.unique(yv, return_counts=True)
    return {int(v): int(c) for v, c in zip(vals, cnts)}

print("Distribución original:", dist(y))
print("Train antes de SMOTE:", dist(y_train))
print("Train después SMOTE:", dist(y_train_sm))
print("Test (sin SMOTE):   ", dist(y_test))


Distribución original: {0: 4920, 1: 492}
Train antes de SMOTE: {0: 3444, 1: 344}
Train después SMOTE: {0: 3444, 1: 1722}
Test (sin SMOTE):    {0: 1476, 1: 148}
